##  This is a script to use deep learning to predict stuff

# Do Imports

In [1]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import time

import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import StandardScaler
#tf.enable_eager_execution()

In [2]:
os.getcwd()

'/home/ec2-user/SageMaker/SYS6016-Final-Project/Code'

# Read data

In [3]:
# Read in data (from pkl file probably)

#os.getcwd()
df = pd.read_pickle('../data/price_level_total_view_2017-01-03_AAPL_grouped_2')
df.head()
df.tail()


,trade_volume,mid_price_log,target,trade_volume_differential,mid_price_log_direction_0_1,mid_price_log_direction_0_2,mid_price_log_direction_0_3,mid_price_log_direction_0_4,mid_price_log_direction_0_5,mid_price_log_direction_0_6,trade_volume_differential_direction_0_1,trade_volume_differential_direction_0_2,trade_volume_differential_direction_0_3,trade_volume_differential_direction_0_4,trade_volume_differential_direction_0_5,trade_volume_differential_direction_0_6
groupid,,,,,,,,,,,,,,,,
57599089,4659293,25.491830,2,0.0,2,2,2,2,2,2,1,1,1,1,1,1
57599090,4659293,25.550179,0,0.0,2,2,2,2,2,2,1,1,1,1,1,1
57599091,4659293,25.372644,2,0.0,0,0,0,0,0,0,1,1,1,1,1,1
57599092,4659293,25.571578,0,0.0,2,2,2,2,2,2,1,1,1,1,1,1
57599093,4659293,24.424568,0,0.0,0,0,0,0,0,0,1,1,1,1,1,1


# Define functions

In [4]:
def features_2d_to_3d(data, labels, window):
    data_n, data_w = data.shape
    stride1, stride2 = data.strides
    new_len = data_n - window
    data3d = as_strided(data, [new_len , window, data_w], strides=[stride1, stride1, stride2])
#    return(data3d, labels[:len(labels)-window])
    return(data3d, labels[window:])

def flatten_3d(data):
    data_n = data.shape[0]
    new_width = data.shape[1]*data.shape[2]
    
    return np.reshape(data, (data_n, new_width)) # flesh this function out
    
def split_data(dfX, dfy, train_frac):

   X = dfX
   y = dfy
   n = X.shape[0]
   cutoff = np.floor(n * train_frac).astype(int) # total - the number you want to test, which here i'm flooring
   #                   (amount you want in training should be 1/10th value the denominator)
   # cutoff

   X_train, X_test = (X.iloc[0:cutoff , :] , X.iloc[cutoff: , :] )
   y_train, y_test = (y.iloc[0:cutoff].values.ravel() , y.iloc[cutoff:].values.ravel() )

   ss = StandardScaler()
   ss.fit(X_train)
   X_train = ss.transform(X_train)
   X_test = ss.transform(X_test)

   return X_train, y_train, X_test, y_test


In [53]:
def create_datasets(trainarray, labelarray, testarray, testlabelarray, window_size):
    tf.reset_default_graph()
    
    train_n = trainarray.shape[0]
    test_n = testarray.shape[0]
    
    #.shuffle(buffer_size=2*train_n)

    with tf.name_scope("dataset"):
        training_dataset = (
            tf.data.Dataset.from_tensor_slices(
                (
                    tf.cast(trainarray, tf.float32)
                )
            ).window(size=window_size,shift=1,stride=1,drop_remainder=False).flat_map(lambda x: x.batch(window_size))
        )
        
        label_dataset = (
            tf.data.Dataset.from_tensor_slices(
                (
                    tf.cast(labelarray, tf.int32)
                )
            ).window(size=1,shift=1,stride=1,drop_remainder=False).flat_map(lambda x: x.batch(1))
        )
        
        training_dataset = tf.data.Dataset.zip((training_dataset, label_dataset))

#         test_dataset = (
#             tf.data.Dataset.from_tensor_slices(
#                 (
#                     tf.cast(testarray, tf.float32)#,
# #                    tf.cast(testlabelarray, tf.int32)
#                 )
#             )
#         ).shuffle(buffer_size=2*test_n).window(size=window_size,shift=1,stride=1,drop_remainder=True)#.flat_map(lambda x: x.batch(window_size))

#    print(training_dataset)
        
    with tf.name_scope("iterator"):
        iterator = tf.data.Iterator.from_structure(training_dataset.output_types, training_dataset.output_shapes)
        features, labels = iterator.get_next()
        train_init = iterator.make_initializer(training_dataset) # initializer for train_data
#         test_init = iterator.make_initializer(test_dataset) # initializer for train_data
        test_init = None

    return features, labels, train_init, test_init

# Define model

In [50]:
def create_rnn_model(features, labels, window_size, batch_size):
    n_outputs = 3
    
    with tf.name_scope("rnn"):
        features = tf.reshape(shape=[-1,14,window_size], tensor=features)
        #labels = tf.reshape(shape=[-1,1], tensor=labels)
        rnn = tf.contrib.rnn.BasicRNNCell(num_units=window_size, name="RNN1")
        init_state = rnn.zero_state(window_size, dtype=tf.float32)
        rnn_outputs, final_state = tf.nn.dynamic_rnn(rnn, features, initial_state=init_state)
    
    with tf.name_scope("output"):
        logits = tf.layers.dense(final_state, n_outputs, name="output")
        Y_proba = tf.nn.softmax(logits, name="Y_proba")

    # Step 5: Define the optimizer; taking as input (learning_rate) and (loss)
    with tf.name_scope("train"):
        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
        loss = tf.reduce_mean(xentropy)
        optimizer = tf.train.AdamOptimizer()
        training_op = optimizer.minimize(loss)

    # Step 6: Define the evaluation metric
    with tf.name_scope("eval"):
        correct = tf.nn.in_top_k(logits, labels, 1)
        accuracy = tf.reduce_sum(tf.cast(correct, tf.float32))

    # Step 7: Initiate
    with tf.name_scope("init_and_save"):
        init = tf.global_variables_initializer()
        saver = tf.train.Saver()
    
    return training_op, loss, accuracy, init

# Create Dataset iterators

In [7]:
# select columns and scale data
dfX = df.reset_index().iloc[:,5:]  #np.delete(np.arange(16), [0,2])
dfX2 = df.reset_index().iloc[:,[2,4]]

ss = StandardScaler()
ss.fit(dfX2)
dfX2 = ss.transform(dfX2)

dfX2 = pd.DataFrame(data=dfX2, columns= ['mid_price_log', 'trade_volume_differential'])
dfX = pd.merge(dfX2, dfX, left_index=True, right_index=True)

dfy = df.reset_index().iloc[:,3]

# Create train/test sets
trainarray,labelarray,testarray,testlabelarray = split_data(dfX, dfy, 0.8)

In [8]:
pd.Series(labelarray).value_counts()

1    115967
2     61580
0     61298
dtype: int64

In [9]:
# create random labels if we want to test accuracy

# labelarray = np.asarray([np.random.randint(0,3) for i in np.arange(labelarray.shape[0])])
# testlabelarray = np.asarray([np.random.randint(0,3) for i in np.arange(testlabelarray.shape[0])])

In [10]:
# transform data set to use data windows
from numpy.lib.stride_tricks import as_strided

window_size = 10
# if window_size isn't even, tf.reshape throws an error down below after pooling
#trainarray, labelarray = features_2d_to_3d(np.array(trainarray), np.array(labelarray), window_size)
#testarray, testlabelarray = features_2d_to_3d(np.array(testarray), np.array(testlabelarray), window_size)
#trainarray = flatten_3d(trainarray)
#testarray = flatten_3d(testarray)

In [11]:
train_n = trainarray.shape[0]
test_n = testarray.shape[0]
#height = trainarray.shape[1]
#width = trainarray.shape[2]
#window_size = 10

batch_size = 400
n_batches = train_n // batch_size
n_batches_test = test_n // batch_size

In [12]:
#trainarray = np.reshape(trainarray, newshape=[-1, height, width, 1])
#testarray = np.reshape(testarray, newshape=[-1, height, width, 1])

In [56]:
# Create datasets & model

features, labels, tr_init, te_init = create_datasets(trainarray, labelarray, testarray, testlabelarray, window_size)
training_op, loss, accuracy, init = create_rnn_model(features, labels, window_size, batch_size)

#temp = create_datasets(trainarray, labelarray, testarray, testlabelarray, window_size)

# Train model

In [58]:
n_epochs = 1

columns = ['t-plus', 'loss', 'accuracy', 'test_loss', 'test_accuracy']
summaries = pd.DataFrame(np.zeros([n_epochs,5], dtype=float), columns=columns)
run_name = 'model1'

writer = tf.summary.FileWriter("./", graph=None)

with tf.Session() as sess:
    start_time = time.time()
    writer.add_graph(sess.graph)
    sess.run(init)
    tot_batches_run = 0
    for epoch in range(n_epochs):
        sess.run(tr_init) # drawing samples from train_data
        train_loss, train_accuracy = 0, 0
        for i in range(n_batches):
            try:
                _, loss_value, acc_value = sess.run([training_op, loss, accuracy]) # , feed_dict={keep_prob : 0.75} # for dropout only
                train_loss += loss_value
                train_accuracy += acc_value
            except tf.errors.OutOfRangeError:
                print("out of range on iter {}".format(i))
                break
        train_accuracy = train_accuracy/(train_n)
                
        # Now get testing loss
        sess.run(te_init) # drawing samples from test_data
        test_loss, test_accuracy = 0, 0
        for i in range(n_batches_test):
            try:
                loss_value, acc_value = sess.run([loss, accuracy]) # , feed_dict={keep_prob : 0.75} # for dropout only
                test_loss += loss_value
                test_accuracy += acc_value
            except tf.errors.OutOfRangeError:
                print("out of range on iter {}".format(i))
                break
        test_accuracy = test_accuracy/(test_n)
        
        
#         epoch_time = time.time()
        print("Epoch: {}, Train_Loss: {:.4f}, Test_Loss: {:.4f}, Train_Accuracy: {:.4f}, Test_Accuracy: {:.4f}"\
              .format(epoch, train_loss, test_loss, train_accuracy, test_accuracy))
#         cum_time = epoch_time - start_time
#         summaries.iloc[epoch,:] = cum_time, train_loss, test_loss, train_accuracy, te_acc


InvalidArgumentError: ConcatOp : Dimensions of inputs should match: shape[0] = [1,14] vs. shape[1] = [10,10]
	 [[node rnn/rnn/while/RNN1/concat (defined at <ipython-input-50-70ee87e9c7d3>:9) ]]
	 [[node eval/Sum (defined at <ipython-input-50-70ee87e9c7d3>:25) ]]

Caused by op 'rnn/rnn/while/RNN1/concat', defined at:
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/kernelapp.p